In [11]:
# %pip freeze > requirements.txt

In [12]:
%pip install -r requirements.txt

In [13]:
# from langchain.agents import initialize_agent, AgentType
from langchain.agents import Tool
from langchain_community.tools import DuckDuckGoSearchRun
from langchain.tools import tool
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from langchain.agents import AgentExecutor, create_react_agent
from langchain.memory import ConversationBufferMemory
from jinja2 import Template

from dotenv import load_dotenv
import json
import os
# import re
import json
import random

In [14]:
load_dotenv()
openai_api_key = os.getenv("OPENAI_API_KEY")

In [15]:
JSON_FILE_PATH = "..\\use_openai\\data\\id_added_dummy_data.json"
# RCMD_PROMPT_PATH = "..\\prompts\\agent_recommend.md"
RCMD_PROMPT_PATH = "..\\prompts\\action_recommend.md"  # ReAct에 맞는 프롬프트로 수정
FEATURE_ID_PATH = "featureId.txt"

In [16]:
# Read prompt
with open(RCMD_PROMPT_PATH, "r", encoding="utf-8", errors="replace") as f:
    RCMD_PROMPT = f.read()

# Read data
with open(JSON_FILE_PATH, 'r', encoding='utf-8') as f:
    data = json.load(f)
    print(type(data))  # (test) <class 'list'>

# Read featureId keys
with open(FEATURE_ID_PATH, "r", encoding="utf-8") as f:
    lines = f.readlines()
    feature_id_list = [line.strip() for line in lines]
    # print(feature_id)  # test

<class 'list'>


In [17]:
# # test
FEATURE_ID = int(random.choice(feature_id_list))
print(FEATURE_ID, type(FEATURE_ID))

6 <class 'int'>


In [18]:
# featureId로 해당 category, feature 이하의 actions만 추출
FEATURE_ID = int(random.choice(feature_id_list))

def extract_workspace_view(input_data, id):
    # featureId로 작업목록 + workspaceId 가져오기
    for item in input_data:
        for category in item.get("recommendedCategories", []):
            for feature in category.get("features", []):
                if feature.get("featureId") == id:
                    return {
                        "workspaceId": item["workspaceId"],
                        "recommendedCategories": [
                            {
                                "categoryId": category["categoryId"],
                                "name": category["name"],
                                "features": [feature]
                            }
                        ]
                    }

# featureId로 읽어온 actions
workspace_data = extract_workspace_view(data, FEATURE_ID)
workspace_data
# print(json.dumps(result, ensure_ascii=False, indent=2))

if workspace_data:
    print(json.dumps(workspace_data, ensure_ascii=False, indent=2))
else:
    print("해당 featureId를 찾을 수 없습니다.")

{
  "workspaceId": 9,
  "recommendedCategories": [
    {
      "categoryId": 4,
      "name": "게시판 기능",
      "features": [
        {
          "featureId": 44,
          "name": "관리 기능",
          "actions": [
            {
              "name": "통계 관리",
              "importance": 5,
              "startDate": "2025-06-15T14:25:34.094111",
              "endDate": "2025-06-26T14:25:34.094111"
            },
            {
              "name": "백업/복구",
              "importance": 3,
              "startDate": "2025-06-14T14:25:34.094114",
              "endDate": "2025-06-20T14:25:34.094114"
            },
            {
              "name": "게시판 권한 관리",
              "importance": 3,
              "startDate": "2025-06-14T14:25:34.094117",
              "endDate": "2025-06-17T14:25:34.094117"
            }
          ]
        }
      ]
    }
  ]
}


In [19]:
type(workspace_data)  # (test) dict

dict

In [20]:
# featureId로 읽어온 actions
workspace_data = extract_workspace_view(data, FEATURE_ID)

# Jinja2 template 객체 생성. 템플릿 구조상(복잡, dict 생성이 필요) 변수 렌더링 필요.
# template = Template(RCMD_PROMPT)

# 템플릿&작업목록 데이터 결합. jinja2 방식 템플릿 렌더링.
# template 내 변수({})들을 실제값으로 치환 -> 문자열 생성
# formatted_input = template.render(ws_data=workspace_data)  # JSON 내 필드만 하나씩 꺼내서 출력
# formatted_input = template.render(ws_data=json.dumps(workspace_data, ensure_ascii=False))   #  JSON 전체를 문자열로 템플릿에 삽입

# print(formatted_input)  # test
# print(type(formatted_input))

In [21]:
# # test
# print(type(template))
# print(template)

In [22]:
# test
search = DuckDuckGoSearchRun()
dump_data = json.dumps(workspace_data, ensure_ascii=False)  # dict -> str
# pattern = r'({[\s\S]*})'
# match = re.search(pattern, dump_data, re.DOTALL)
# json_block = match.group(0)
print(type(dump_data))
print(dump_data)

<class 'str'>
{"workspaceId": 9, "recommendedCategories": [{"categoryId": 4, "name": "게시판 기능", "features": [{"featureId": 44, "name": "관리 기능", "actions": [{"name": "통계 관리", "importance": 5, "startDate": "2025-06-15T14:25:34.094111", "endDate": "2025-06-26T14:25:34.094111"}, {"name": "백업/복구", "importance": 3, "startDate": "2025-06-14T14:25:34.094114", "endDate": "2025-06-20T14:25:34.094114"}, {"name": "게시판 권한 관리", "importance": 3, "startDate": "2025-06-14T14:25:34.094117", "endDate": "2025-06-17T14:25:34.094117"}]}]}]}


In [23]:
# OpenAI 객체 초기화
llm = ChatOpenAI(
    model="gpt-4o-mini",
    temperature=0.3,
    max_tokens=1024,
    openai_api_key=openai_api_key
)

In [24]:
RCMD_PROMPT

'당신은 JSON 기반의 워크스페이스 작업 흐름을 분석하여, 각 기능(feature)에 대해 적절한 후속 작업(action)을 제안하는 전문가입니다.\n\n당신의 임무는 다음과 같습니다:\n\n1. 입력으로 주어진 JSON에서 workspaceId, category, feature, action 구조를 파악하고,\n2. 각 feature에 대해 3개의 후속 작업(action)을 생성하며,\n3. 이 결과를 지정된 형식의 JSON으로 반환하는 것입니다.\n\n---\n\n입력 정보:\n\n- 입력 JSON: {input}\n- 사용 가능한 도구 목록: {tools}\n- 사용 가능한 도구 이름: {tool_names}\n- 이전 실행 이력 (agent_scratchpad): {agent_scratchpad}\n\n---\n\n아래의 ReAct 형식 규칙을 반드시 따르십시오. 하나라도 어기면 시스템이 응답을 인식하지 못하고 실패합니다.\n\nReAct 실행 구조:\n\n1. Thought: 현재 해야 할 판단과 적절한 도구 선택 이유를 서술하십시오.\n2. Action: 사용할 도구의 이름을 정확히 명시하십시오 (예: Action: ParseWorkflow). 반드시 tool_names에 있는 이름과 일치해야 하며, 대소문자도 구분됩니다.\n3. Action Input: 선택한 도구에 전달할 입력 데이터를 JSON 형식으로 작성하십시오.\n4. Observation: 도구 실행 결과를 간단히 요약하십시오.\n5. Final Answer: 반드시 FinalAnswer 도구를 사용하여 JSON 출력 결과를 전달하십시오. JSON을 직접 출력하지 마십시오. Final Answer 뒤에는 Thought가 다시 나와서는 안 됩니다.\n\n---\n\n실행 순서:\n\n1. ParseWorkflow 도구를 먼저 실행하여 각 기능에 대해 3개의 후속 작업을 생성하십시오.\n2. 그 결과를 FinalAnswer 도구를 사용해 출력하십시오.\n'

In [25]:
# template

In [26]:
# 변수{} 삽입 가능한 템플릿 생성
prompt = PromptTemplate(
    input_variables=["input", "tools", "tool_names", "agent_scratched"],
    template=RCMD_PROMPT
    # template=template
)

In [ ]:
# llm(OpenAI) <-> Langchain 연결
llm_chain = LLMChain(prompt=prompt, llm=llm)

# LLM으로 action 추천 받는 함수
def generate_next_actions(s_result):
    return llm_chain.run(s_result)

# 전체 JSON을 받아 feature별 추천 작업을 생성하는 메인 함수
# @tool(description="워크스페이스 JSON으로부터 feature별 추천 작업을 생성합니다.")
def parse_and_generate_all(input_str: dict) -> str:

    data = input_str  # 수정!!!!!!!

    # 웹 검색 툴 정의
    search = DuckDuckGoSearchRun()

    for category in data.get("recommendedCategories", []):  # recommendedCategories 가져오기
        for feature in category.get("features", []):        # recommendedCategories 내부의 feature에서

            # 웹 검색
            try:
                search_query = f"{feature['name']} 기능 구현 방법"  # feature name에 대한 actions 서치
                search_result = search.run(search_query)
            except Exception as e:
                search_result = f"** 웹 검색 실패: {e} **"
                
            # featureName, existingActions 변수 사용 안 하고 서치
            llm_input = search_result[:500]
            
            generated_actions = []

            try:
                generated_json_str = generate_next_actions(llm_input)  # 추천 actions 생성
                generated_actions = json.loads(generated_json_str)     # 추천 actions를 json형식의 python 객체로 load
                feature["actions"].extend(generated_actions)
            except Exception as e:
                feature["actions"].append({
                    "name": f"작업 생성 실패: {e}",
                    "startDate": "2025-01-01T00:00:00",
                    "endDate": "2025-01-01T23:59:59",
                    "importance": 0
                })

    # return json.dumps(data, ensure_ascii=False, indent=2)
    return data

C:\Users\user\AppData\Local\Temp\ipykernel_25384\3223574271.py:2: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  llm_chain = LLMChain(prompt=prompt, llm=llm)


In [28]:
parse_and_generate_all(workspace_data)  # test

C:\Users\user\AppData\Local\Temp\ipykernel_25384\3223574271.py:6: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  return llm_chain.run(input_dict)


{'workspaceId': 9,
 'recommendedCategories': [{'categoryId': 4,
   'name': '게시판 기능',
   'features': [{'featureId': 44,
     'name': '관리 기능',
     'actions': [{'name': '통계 관리',
       'importance': 5,
       'startDate': '2025-06-15T14:25:34.094111',
       'endDate': '2025-06-26T14:25:34.094111'},
      {'name': '백업/복구',
       'importance': 3,
       'startDate': '2025-06-14T14:25:34.094114',
       'endDate': '2025-06-20T14:25:34.094114'},
      {'name': '게시판 권한 관리',
       'importance': 3,
       'startDate': '2025-06-14T14:25:34.094117',
       'endDate': '2025-06-17T14:25:34.094117'},
      {'name': "작업 생성 실패: Missing some input keys: {'input', 'tools', 'tool_names'}",
       'startDate': '2025-01-01T00:00:00',
       'endDate': '2025-01-01T23:59:59',
       'importance': 0}]}]}]}

In [29]:
# 툴 정의
# 파싱용(action 추천)
parse_tool = Tool(
    name="ParseWorkflow",
    func=parse_and_generate_all,
    description="JSON 작업 흐름에 대해 feature별로 다음 추천 작업(actions) 3개를 추가합니다."
)

# 출력용
final_tool = Tool(
    name="FinalAnswer",
    func=lambda x: x,  # 결과를 그대로 반환
    description="최종 JSON 결과를 반환합니다.",
    return_direct=True  # 도구 호출 즉시 결과 반환, agent 종료 
)

tools = [parse_tool, final_tool]

In [30]:
# 먼저 에이전트 생성하고
# create_react_agent()는 initialize_agent() 보다 세밀한 설정 가능
agent = create_react_agent(
    llm=llm,
    tools=tools,  # 단순 tools 선언, AgentExecutor에서 tools 재전달 필요
    prompt=prompt  # 프롬프트 여기서 넘김
)

# 메모리 설정
memory = ConversationBufferMemory(
    memory_key="agent_scratchpad",   # 대화 로그 저장용
    input_key="input"                # 사람이 말한 내용이 들어가는 키
)

# 다음에 AgentExecutor로 루프 반복 컨트롤?
agent_executor = AgentExecutor(
    agent=agent,
    tools=tools,  # tools의 기능 전달
    memory=memory,
    max_iterations=3,
    max_execution_time=2,
    early_stopping_method="force",  # default="force"
    handle_parsing_errors=True,
    verbose=True
)

C:\Users\user\AppData\Local\Temp\ipykernel_25384\1183251216.py:10: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory(


In [31]:
# # tests
# prompt_filled = RCMD_PROMPT.replace("{input}", json.dumps(workspace_data, ensure_ascii=False))  # 위에서 이미 함
# prompt_filled

In [32]:
tool_names = [tool.name for tool in tools]

response = agent_executor.invoke({
    "input": json.dumps(workspace_data, ensure_ascii=False),
    "tool_names": ["ParseWorkflow", "Final Answer"],
    "tool_names": tool_names,
    "agent_scratchpad": ""
})

# 답변 생성 확인
print(response)



> Entering new AgentExecutor chain...
Parsing LLM output produced both a final answer and a parse-able action:: 1. Thought: 입력 JSON에서 "게시판 기능"의 "관리 기능"에 대한 후속 작업을 생성해야 합니다. 이 작업을 위해 ParseWorkflow 도구를 사용하여 각 feature에 대해 3개의 후속 작업을 추가할 것입니다. 
2. Action: ParseWorkflow
3. Action Input: {"workspaceId": 9, "recommendedCategories": [{"categoryId": 4, "name": "게시판 기능", "features": [{"featureId": 44, "name": "관리 기능", "actions": [{"name": "통계 관리", "importance": 5, "startDate": "2025-06-15T14:25:34.094111", "endDate": "2025-06-26T14:25:34.094111"}, {"name": "백업/복구", "importance": 3, "startDate": "2025-06-14T14:25:34.094114", "endDate": "2025-06-20T14:25:34.094114"}, {"name": "게시판 권한 관리", "importance": 3, "startDate": "2025-06-14T14:25:34.094117", "endDate": "2025-06-17T14:25:34.094117"}, {"name": "작업 생성 실패: Missing some input keys: {'input', 'tools', 'tool_names'}", "startDate": "2025-01-01T00:00:00", "endDate": "2025-01-01T23:59:59", "importance": 0}]}]}]}
4. Observation: ParseWorkflow 도구가 실행되

In [33]:
response

{'input': '{"workspaceId": 9, "recommendedCategories": [{"categoryId": 4, "name": "게시판 기능", "features": [{"featureId": 44, "name": "관리 기능", "actions": [{"name": "통계 관리", "importance": 5, "startDate": "2025-06-15T14:25:34.094111", "endDate": "2025-06-26T14:25:34.094111"}, {"name": "백업/복구", "importance": 3, "startDate": "2025-06-14T14:25:34.094114", "endDate": "2025-06-20T14:25:34.094114"}, {"name": "게시판 권한 관리", "importance": 3, "startDate": "2025-06-14T14:25:34.094117", "endDate": "2025-06-17T14:25:34.094117"}, {"name": "작업 생성 실패: Missing some input keys: {\'input\', \'tools\', \'tool_names\'}", "startDate": "2025-01-01T00:00:00", "endDate": "2025-01-01T23:59:59", "importance": 0}]}]}]}',
 'tool_names': ['ParseWorkflow', 'FinalAnswer'],
 'agent_scratchpad': '',
 'output': 'Agent stopped due to iteration limit or time limit.'}